# Diagnose Issues for Level 4 basin 2321, Garonne et al.

October 10 and 11, Mike D

In [ ]:
import json
import copy
from pathlib import Path
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
from netCDF4 import Dataset
import random
import webbrowser
import matplotlib.pyplot as plt
import os
from check_topology import check_topology
from plotIssueMap import plotIssueMap
from apply_sword_patches import apply_sword_patches
from datetime import datetime

In [ ]:
os.getcwd()

In [ ]:
# working directory
InputDir=Path('/Users/mtd/Data/SWOT/SWORD/SWORD_v15/Reaches_Nodes/netcdf')
ThisDir=Path('./')

In [ ]:
# choose the basin Id to look at (each basin file contains multiple L3 basins)
Lxbasin='2321'

# Read and set up SWORD data

For all data in the specified basin, pass the data into a dictionary to be used throughout.

In [ ]:
#pull out the SWORD data. create a dictionary keyed off reach data with centerline and reach properties

swordfile=InputDir.joinpath('eu_sword_v15.nc')
sword_dataset=Dataset(swordfile)

sword_point_reachids=sword_dataset['centerlines/reach_id'][0,:][:]
swordx=sword_dataset['centerlines/x'][:]
swordy=sword_dataset['centerlines/y'][:]

swordreachids=sword_dataset["reaches/reach_id"][:].tolist()
sword_names=sword_dataset['reaches/river_name'][:]
sword_drainage_area=sword_dataset['reaches/facc'][:]
sword_swot_orbits=sword_dataset['reaches/swot_orbits'][:]

sword_rch_id_up=sword_dataset['reaches/rch_id_up'][:]
sword_rch_id_dn=sword_dataset['reaches/rch_id_dn'][:]

sword_n_rch_up=sword_dataset['reaches/n_rch_up'][:]
sword_n_rch_down=sword_dataset['reaches/n_rch_down'][:]


sword_dataset.close()

In [ ]:
BasinLevel=len(Lxbasin)

domain_reachids=[]
for reachid in swordreachids:
    reachidstr=str(reachid)
    if reachidstr[0:BasinLevel] == Lxbasin:
        domain_reachids.append(reachid)

print('there are a total of ',len(domain_reachids),'reaches in SWORD for', Lxbasin)

In [ ]:
# create a dictionary keyed off reach data with centerline and reach properties for domain: 
# this takes a second

domain_reach_data={}

for reach in domain_reachids:
    
    reachstr=str(reach)
    
    # deal with points
    indxs=np.argwhere(sword_point_reachids.data==int(reach))   
    indxs=indxs[:,0]
    points=[]
    for indx in indxs:           
        points.append(tuple([swordy[indx],swordx[indx]]))    
        
    # deal with reaches
    indx = swordreachids.index(int(reach))
    
    domain_reach_data[reachstr]={}
    domain_reach_data[reachstr]['clpoints']=points
    domain_reach_data[reachstr]['river_name']=sword_names[indx]
    
    domain_reach_data[reachstr]['drainage_area_km2']=sword_drainage_area[indx]
    domain_reach_data[reachstr]['swot_orbits']=sword_swot_orbits[:,indx]
    

    domain_reach_data[reachstr]['rch_id_up']=sword_rch_id_up[:,indx]
    domain_reach_data[reachstr]['rch_id_dn']=sword_rch_id_dn[:,indx]    
    
    domain_reach_data[reachstr]['n_rch_up']=sword_n_rch_up[indx]
    domain_reach_data[reachstr]['n_rch_down']=sword_n_rch_down[indx]    


# Set up current patch data

Load, and then setup metadata

In [ ]:
# load current patch file
init_patch_json=ThisDir.joinpath('sword_patches_v215_v2.json')
with open(init_patch_json) as json_file:
    patch_data = json.load(json_file)

patch_data

In [ ]:
today = datetime.now()

patch_data['metadata']['version_num']+=1
patch_data['metadata']['most recent update:']=today.strftime("%Y-%m-%d")
patch_data['metadata']

In [ ]:
# apply patch data though it does not change anything here
domain_reach_data=apply_sword_patches(patch_data,domain_reach_data)

# Run Automated Topology Check

In [ ]:
#
Verbose=1
nerrors,reaches_with_issues=check_topology(domain_reachids,domain_reach_data,Verbose)

print('There are a total of ',nerrors[1],'type 1 errors')
print('There are a total of ',nerrors[2],'type 2 errors')

# Untangling topology issues above

## Issue 1: 

* Issue flagged for 23212000964
* 23212000971 might need fixed: just update reach_id_dn to 23212000964
* 23212000964 might need fixed: update to be between #7 and #5, fix numbers
* 23212000954 might need fixed: update to be between #6 and #4, fix numbers
* 23212000944 might need fixed: update to be within #5 and #3, fix numbers
* 23212000931 might need fixed: update to be within #4 and #2, fix numbers

Some of thsoe may be self-consistent. Quicker to just make it right than to stick with what's there.

In [ ]:
L4basin='2321'
reachstr='23212000964'
reachmap=plotIssueMap(L4basin,domain_reach_data,reachstr,domain_reachids)
reachmap

In [ ]:
# create patch
reachid='23212000971'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23212000964, 0, 0, 0]

In [ ]:
# create patch
reachid='23212000964'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23212000971, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23212000954, 0, 0, 0]

In [ ]:
# create patch
reachid='23212000954'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23212000964, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23212000944, 0, 0, 0]

In [ ]:
# create patch
reachid='23212000944'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23212000954, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23212000931, 0, 0, 0]

In [ ]:
# create patch
reachid='23212000931'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23212000944, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23212000924, 0, 0, 0]

In [ ]:
# apply patches 
domain_reach_data_fix=apply_sword_patches(patch_data,domain_reach_data)

In [ ]:
Verbose=1 #show warnings too!
nerrors,reaches_with_issues=check_topology(domain_reachids,domain_reach_data_fix,Verbose)

print('There are a total of ',nerrors[1],'type 1 errors')
print('There are a total of ',nerrors[2],'type 2 errors')

Yay 23212000964 is no longer in the list

## Issue 2

Flagged 23212001164

Very similar to previous - long string of reaches that all need fixed and set straight in line:
* 23212001171
* 23212001164
* 23212001154
* 23212000584
* 23212000574

In [ ]:
L4basin='2321'
reachstr='23212001164'
reachmap=plotIssueMap(L4basin,domain_reach_data,reachstr,domain_reachids)
reachmap

In [ ]:
# create patch
reachid='23212001171'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['rch_id_dn']=[23212001164, 0, 0, 0]

In [ ]:
# create patch
reachid='23212001164'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23212001171, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23212001154, 0, 0, 0]

In [ ]:
# create patch
reachid='23212001154'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23212001164, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23212000584, 0, 0, 0]

In [ ]:
# create patch
reachid='23212000584'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23212001154, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23212000574, 0, 0, 0]

In [ ]:
# create patch
reachid='23212000574'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23212000584, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23212000561, 0, 0, 0]

In [ ]:
# apply patches 
domain_reach_data_fix=apply_sword_patches(patch_data,domain_reach_data)

In [ ]:
Verbose=1 #show warnings too!
nerrors,reaches_with_issues=check_topology(domain_reachids,domain_reach_data_fix,Verbose)

print('There are a total of ',nerrors[1],'type 1 errors')
print('There are a total of ',nerrors[2],'type 2 errors')

That resolved 23212001164 yay.

## Issue 3

23212001185 is flagged
* 23212001406 is fine
* 23212001185 is actually fine 
* 23212001205 needs fixed: should point down to 23212001185,and point up to 23212001214


In [ ]:
reachstr='23212001185'
reachmap=plotIssueMap(L4basin,domain_reach_data,reachstr,domain_reachids)
reachmap

In [ ]:
# create patch
reachid='23212001205'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['rch_id_dn']=[23212001185, 0, 0, 0]
patch_data['reach_data'][reachid]['rch_id_up']=[23212001214, 0, 0, 0]

In [ ]:
# apply patches 
domain_reach_data_fix=apply_sword_patches(patch_data,domain_reach_data)

In [ ]:
Verbose=1 #show warnings too!
nerrors,reaches_with_issues=check_topology(domain_reachids,domain_reach_data_fix,Verbose)

print('There are a total of ',nerrors[1],'type 1 errors')
print('There are a total of ',nerrors[2],'type 2 errors')

## Issue 4

23212001416 is flagged

Ah there is a weird ghost there. I can just hop over it, I think.

In [ ]:
reachstr='23212001416'
reachmap=plotIssueMap(L4basin,domain_reach_data,reachstr,domain_reachids)
reachmap

In [ ]:
# create patch
reachid='23212001416'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=0
patch_data['reach_data'][reachid]['rch_id_up']=[0, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=0
patch_data['reach_data'][reachid]['rch_id_dn']=[0, 0, 0, 0]


In [ ]:
# apply patches 
domain_reach_data_fix=apply_sword_patches(patch_data,domain_reach_data)

In [ ]:
Verbose=1 #show warnings too!
nerrors,reaches_with_issues=check_topology(domain_reachids,domain_reach_data_fix,Verbose)

print('There are a total of ',nerrors[1],'type 1 errors')
print('There are a total of ',nerrors[2],'type 2 errors')

Yay! 23212001416 is no longer in problem list

## Issue 5

23213000106 is flagged

* 23213000106 needs fixed" just need to set the n_rch_down to 0

In [ ]:
reachstr='23213000106'
reachmap=plotIssueMap(L4basin,domain_reach_data,reachstr,domain_reachids)
reachmap

In [ ]:
# create patch
reachid='23213000106'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_down']=0


In [ ]:
# apply patches 
domain_reach_data_fix=apply_sword_patches(patch_data,domain_reach_data)

In [ ]:
Verbose=1 #show warnings too!
nerrors,reaches_with_issues=check_topology(domain_reachids,domain_reach_data_fix,Verbose)

print('There are a total of ',nerrors[1],'type 1 errors')
print('There are a total of ',nerrors[2],'type 2 errors')

Yay! 23213000106 is no longer in problem list

## Issue 6

23214200404 is flagged, saying 23214200411 as an unrequited downstream neighbor

* Upstream: Looks like 23214200451 is good, flowing the right direction. 
* Downstream: Looks like 23214200384 is good, flowing the right direction etc. 


All reaches in between need fixed: easier to fix 'em all rather than make changes consistent with SWORD

* 23214200444
* 23214200434
* 23214200424
* 23214200411
* 23214200404
* 23214200391

Just line em up & knock em down!


In [ ]:
reachstr='23214200404'
reachmap=plotIssueMap(L4basin,domain_reach_data,reachstr,domain_reachids)
reachmap

In [ ]:
# create patch
reachid='23214200444'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23214200451, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23214200434, 0, 0, 0]


In [ ]:
# create patch
reachid='23214200434'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23214200444, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23214200424, 0, 0, 0]


In [ ]:
# create patch
reachid='23214200424'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23214200434, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23214200411, 0, 0, 0]


In [ ]:
# create patch
reachid='23214200411'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23214200424, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23214200404, 0, 0, 0]


In [ ]:
# create patch
reachid='23214200404'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23214200411, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23214200391, 0, 0, 0]


In [ ]:
# create patch
reachid='23214200391'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23214200404, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23214200384, 0, 0, 0]


In [ ]:
# apply patches 
domain_reach_data_fix=apply_sword_patches(patch_data,domain_reach_data)

In [ ]:
Verbose=1 #show warnings too!
nerrors,reaches_with_issues=check_topology(domain_reachids,domain_reach_data_fix,Verbose)

print('There are a total of ',nerrors[1],'type 1 errors')
print('There are a total of ',nerrors[2],'type 2 errors')

print('Is reach of interest still in reaches with issues list?',  '23214200404' in reaches_with_issues)


**Yay! Issue 6 closed!**

## Issue 7

23214300223 is flagged

Ok it's a one point reach. Can fix by just disconnecting from other reaches.

In [ ]:
reachstr='23214300223'
reachmap=plotIssueMap(L4basin,domain_reach_data,reachstr,domain_reachids)
reachmap

In [ ]:
domain_reach_data['23214300223']

In [ ]:
# create patch
reachid='23214300223'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=0
patch_data['reach_data'][reachid]['rch_id_up']=[0, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=0
patch_data['reach_data'][reachid]['rch_id_dn']=[0, 0, 0, 0]


In [ ]:
# apply patches 
domain_reach_data_fix=apply_sword_patches(patch_data,domain_reach_data)

In [ ]:
Verbose=1 #show warnings too!
nerrors,reaches_with_issues=check_topology(domain_reachids,domain_reach_data_fix,Verbose)

print('There are a total of ',nerrors[1],'type 1 errors')
print('There are a total of ',nerrors[2],'type 2 errors')
print('Is reach of interest still in reaches with issues list?',  'xxxx' in reaches_with_issues)


**Yay! Issue 7 closed!**

## Issue 8

23216000714 is flagged: 23216000714 says 23216000721 is a downstream neighbor, but the neighbor designation is unrequited

Looks like the river is running the right direction overall

* 23216000721 looks ok (upstream)
* 23216000714 needs fixed
* 23216000854 needs fixed
* 23216000841 looks ok (downstream end)

In [ ]:
reachstr='23216000714'
reachmap=plotIssueMap(L4basin,domain_reach_data,reachstr,domain_reachids)
reachmap

In [ ]:
# create patch
reachid='23216000714'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23216000721, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23216000854, 0, 0, 0]


In [ ]:
# create patch
reachid='23216000854'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23216000714, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23216000841, 0, 0, 0]


In [ ]:
# apply patches 
domain_reach_data_fix=apply_sword_patches(patch_data,domain_reach_data)

In [ ]:
Verbose=1 #show warnings too!
nerrors,reaches_with_issues=check_topology(domain_reachids,domain_reach_data_fix,Verbose)

print('There are a total of ',nerrors[1],'type 1 errors')
print('There are a total of ',nerrors[2],'type 2 errors')
print('Is reach of interest still in reaches with issues list?',  'xxxx' in reaches_with_issues)


**Yay! Issue 8 closed!**

## Issue 9

23216000781 says 23216000794 is an upstream neighbor, but the neighbor designation is unrequited


In [ ]:
reachstr='23216000781'
reachmap=plotIssueMap(L4basin,domain_reach_data,reachstr,domain_reachids)
reachmap

### Analysis

* River seems to be running in right direction overall
* 23216000874 seems ok (upstream end)
* 23216000861 needs fixed
* 23216000794 needs fixed
* 23216000781 seems ok (downstream end)

In [ ]:
# create patch
reachid='23216000861'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23216000874, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23216000794 , 0, 0, 0]


In [ ]:
# create patch
reachid='23216000794'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23216000861, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23216000781 , 0, 0, 0]


In [ ]:
# apply patches 
domain_reach_data_fix=apply_sword_patches(patch_data,domain_reach_data)

In [ ]:
Verbose=1 #show warnings too!
nerrors,reaches_with_issues=check_topology(domain_reachids,domain_reach_data_fix,Verbose)

print('There are a total of ',nerrors[1],'type 1 errors')
print('There are a total of ',nerrors[2],'type 2 errors')
print('Is reach of interest still in reaches with issues list?',  '23216000781' in reaches_with_issues)


**Yay! Issue n closed!**

Although, another thing has cropped up, so error count stays the same :(

## Issue 10

23216000011 says 23216000861 is an upstream neighbor, but the neighbor designation is unrequited

In [ ]:
reachstr='23216000011'
reachmap=plotIssueMap(L4basin,domain_reach_data,reachstr,domain_reachids)
reachmap

In [ ]:
domain_reach_data['23216000011']

### Analysis

23216000011 seems to be a phantom reach. I'll just disconnect it from its two upsteram connections, and we should be good.

In [ ]:
# create patch
reachid='23216000011'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=0
patch_data['reach_data'][reachid]['rch_id_up']=[0, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=0
patch_data['reach_data'][reachid]['rch_id_dn']=[0, 0, 0, 0]


In [ ]:
# apply patches 
domain_reach_data_fix=apply_sword_patches(patch_data,domain_reach_data)

In [ ]:
Verbose=1 #show warnings too!
nerrors,reaches_with_issues=check_topology(domain_reachids,domain_reach_data_fix,Verbose)

print('There are a total of ',nerrors[1],'type 1 errors')
print('There are a total of ',nerrors[2],'type 2 errors')
print('Is reach of interest still in reaches with issues list?',  '23216000781' in reaches_with_issues)


**Yay! Issue 10 closed!**

## Issue 11

23216000831 says 23216000841 is a downstream neighbor, but the neighbor designation is unrequited

In [ ]:
reachstr='23216000831'
reachmap=plotIssueMap(L4basin,domain_reach_data,reachstr,domain_reachids)
reachmap

### Analysis

Looks like just 23216000831 needs fixed

In [ ]:
# create patch
reachid='23216000831'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23216000841, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23216001565, 0, 0, 0]


In [ ]:
# apply patches 
domain_reach_data_fix=apply_sword_patches(patch_data,domain_reach_data)

In [ ]:
Verbose=1 #show warnings too!
nerrors,reaches_with_issues=check_topology(domain_reachids,domain_reach_data_fix,Verbose)

print('There are a total of ',nerrors[1],'type 1 errors')
print('There are a total of ',nerrors[2],'type 2 errors')
print('Is reach of interest still in reaches with issues list?',  '23216000831' in reaches_with_issues)


**Yay! Issue 11 resolved!**

## Issue 12

23218000534 says 23218000524 is a downstream neighbor, but the neighbor designation is unrequited

In [ ]:
reachstr='23218000534'
reachmap=plotIssueMap(L4basin,domain_reach_data,reachstr,domain_reachids)
reachmap

### Analysis

* 23218000151 looks good on the upstream end
* 23218000574 looks ok
* 23218000561 needs fixed
* 23218000554 needs fixed
* 23218000544 needs fixed
* 23218000534 needs fixed
* 23218000524 needs fixed
* 23218000511 looks good on downstream end

In [ ]:
# create patch
reachid='23218000561'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23218000574, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23218000554, 0, 0, 0]

In [ ]:
# create patch
reachid='23218000554'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23218000561, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23218000544, 0, 0, 0]

In [ ]:
# create patch
reachid='23218000544'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23218000554, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23218000534, 0, 0, 0]

In [ ]:
# create patch
reachid='23218000534'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23218000544, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23218000524, 0, 0, 0]

In [ ]:
# create patch
reachid='23218000524'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=1
patch_data['reach_data'][reachid]['rch_id_up']=[23218000534, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=1
patch_data['reach_data'][reachid]['rch_id_dn']=[23218000511, 0, 0, 0]

In [ ]:
# apply patches 
domain_reach_data_fix=apply_sword_patches(patch_data,domain_reach_data)

In [ ]:
Verbose=1 #show warnings too!
nerrors,reaches_with_issues=check_topology(domain_reachids,domain_reach_data_fix,Verbose)

print('There are a total of ',nerrors[1],'type 1 errors')
print('There are a total of ',nerrors[2],'type 2 errors')
print('Is reach of interest still in reaches with issues list?',  '23216000781' in reaches_with_issues)


**Yay! All issues closed!**

# Write out final patch file



In [ ]:
patch_data['metadata']

In [ ]:
# write to json file
patch_file='sword_patches_v215_v3.json'
with open(patch_file,'w') as json_file:
     json.dump(patch_data,json_file, indent=2)

# Template

Copy and paste these pieces up into each new issue...

## Issue n

xxxx is flagged

In [ ]:
reachstr='xxxx'
reachmap=plotIssueMap(L4basin,domain_reach_data,reachstr,domain_reachids)
reachmap

### Analysis

* xxxx needs fixed...

In [ ]:
# create patch
reachid='xxxx'

patch_data['reach_data'][reachid]={    
}

# the first entry in each reach or node entry is a metadata dictionary
patch_data['reach_data'][reachid]['metadata']={
    'date_patch_added':'2023-10-10',
    'person_responsible':'Mike Durand',
    'reason_for_patch':'bad topology',
    'patch_comment':'n/a'
}

# data elements to patch
patch_data['reach_data'][reachid]['n_rch_up']=0
patch_data['reach_data'][reachid]['rch_id_up']=[0, 0, 0, 0]
patch_data['reach_data'][reachid]['n_rch_down']=0
patch_data['reach_data'][reachid]['rch_id_dn']=[0, 0, 0, 0]


In [ ]:
# apply patches 
domain_reach_data_fix=apply_sword_patches(patch_data,domain_reach_data)

In [ ]:
Verbose=1 #show warnings too!
nerrors,reaches_with_issues=check_topology(domain_reachids,domain_reach_data_fix,Verbose)

print('There are a total of ',nerrors[1],'type 1 errors')
print('There are a total of ',nerrors[2],'type 2 errors')
print('Is reach of interest still in reaches with issues list?',  '23216000781' in reaches_with_issues)


**Yay! Issue n closed!**